# Shipping Cost Prediction


### Reg No: IT21134180
### Name: Vihansa S.A.S

<hr/>

<ul>
    <li><b>Target Variable:</b> Shipping Cost </li>
    <li><b>Predictors:</b> Type here the predictors...</li>
    <li><b>Objective:</b> Type here the objective.</li>
</ul>




In [65]:
#Importing required libraries
import seaborn as sns
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


In [66]:
#Import the dataset 
dataset = pd.read_csv('../dataset/SuperStoreOrders.csv')

In [67]:
# Visualise the dataset 
dataset

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.1400,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.0360,9.72,Medium,2011
2,HU-2011-1220,1/1/2011,5/1/2011,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,...,Office Supplies,Storage,"Tenex Box, Single Width",66,4,0.0,29.6400,8.17,High,2011
3,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,...,Office Supplies,Paper,"Enermax Note Cards, Premium",45,3,0.5,-26.0550,4.82,High,2011
4,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",114,5,0.1,37.7700,4.70,Medium,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,CA-2014-115427,31-12-2014,4/1/2015,Standard Class,Erica Bern,Corporate,California,United States,US,West,...,Office Supplies,Binders,"Cardinal Slant-D Ring Binder, Heavy Gauge Vinyl",14,2,0.2,4.5188,0.89,Medium,2014
51286,MO-2014-2560,31-12-2014,5/1/2015,Standard Class,Liz Preis,Consumer,Souss-Massa-Draâ,Morocco,Africa,Africa,...,Office Supplies,Binders,"Wilson Jones Hole Reinforcements, Clear",4,1,0.0,0.4200,0.49,Medium,2014
51287,MX-2014-110527,31-12-2014,2/1/2015,Second Class,Charlotte Melton,Consumer,Managua,Nicaragua,LATAM,Central,...,Office Supplies,Labels,"Hon Color Coded Labels, 5000 Label Set",26,3,0.0,12.3600,0.35,Medium,2014
51288,MX-2014-114783,31-12-2014,6/1/2015,Standard Class,Tamara Dahlen,Consumer,Chihuahua,Mexico,LATAM,North,...,Office Supplies,Labels,"Hon Legal Exhibit Labels, Alphabetical",7,1,0.0,0.5600,0.20,Medium,2014


In [68]:
#Select columns in the dataset 
dataset = dataset[["order_date","ship_mode","segment","state","market","region","category","sales","quantity","discount","order_priority","shipping_cost"]]
                
dataset

,order_date,ship_mode,segment,state,market,region,category,sales,quantity,discount,order_priority,shipping_cost
0,1/1/2011,Standard Class,Consumer,Constantine,Africa,Africa,Office Supplies,408,2,0.0,Medium,35.46
1,1/1/2011,Standard Class,Consumer,New South Wales,APAC,Oceania,Office Supplies,120,3,0.1,Medium,9.72
2,1/1/2011,Second Class,Consumer,Budapest,EMEA,EMEA,Office Supplies,66,4,0.0,High,8.17
3,1/1/2011,Second Class,Home Office,Stockholm,EU,North,Office Supplies,45,3,0.5,High,4.82
4,1/1/2011,Standard Class,Consumer,New South Wales,APAC,Oceania,Furniture,114,5,0.1,Medium,4.70
...,...,...,...,...,...,...,...,...,...,...,...,...
51285,31-12-2014,Standard Class,Corporate,California,US,West,Office Supplies,14,2,0.2,Medium,0.89
51286,31-12-2014,Standard Class,Consumer,Souss-Massa-Draâ,Africa,Africa,Office Supplies,4,1,0.0,Medium,0.49
51287,31-12-2014,Second Class,Consumer,Managua,LATAM,Central,Office Supplies,26,3,0.0,Medium,0.35
51288,31-12-2014,Standard Class,Consumer,Chihuahua,LATAM,North,Office Supplies,7,1,0.0,Medium,0.20


In [69]:
#Checking for Missing values 
dataset.isnull() #It is difficult to identify missing values using this method since the dataset is large.
dataset.isnull().values.any() #Using this command we can scan the entire dataset and get a verification whether there are missing values or not 

False

In [70]:
#Getting the number of categoricaal variables of each feature seperately.
dataset['ship_mode'].value_counts(normalize=True)

ship_mode
Standard Class    0.600019
Second Class      0.200994
First Class       0.146325
Same Day          0.052661
Name: proportion, dtype: float64

In [71]:
#Get the count of variables that has unique values in the dataset 
categorical_counts = dataset.select_dtypes(include=['object']).nunique()
print(categorical_counts)

order_date        1430
ship_mode            4
segment              3
state             1094
market               7
region              13
category             3
sales             2246
order_priority       4
dtype: int64


In [72]:
# Initialise label encoder
label_encoder = LabelEncoder() 
  
# Encode labels in the data frame. 
dataset['ship_mode']= label_encoder.fit_transform(dataset['ship_mode']) 
dataset['segment']= label_encoder.fit_transform(dataset['segment']) 
dataset['state']= label_encoder.fit_transform(dataset['state']) 
dataset['market']= label_encoder.fit_transform(dataset['market']) 
dataset['region']= label_encoder.fit_transform(dataset['region']) 
dataset['category']= label_encoder.fit_transform(dataset['category']) 
dataset['order_priority']= label_encoder.fit_transform(dataset['order_priority']) 
#Print the data frame
dataset

C:\Users\subas\AppData\Local\Temp\ipykernel_6984\2408156750.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['ship_mode']= label_encoder.fit_transform(dataset['ship_mode'])
C:\Users\subas\AppData\Local\Temp\ipykernel_6984\2408156750.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['segment']= label_encoder.fit_transform(dataset['segment'])
C:\Users\subas\AppData\Local\Temp\ipykernel_6984\2408156750.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

,order_date,ship_mode,segment,state,market,region,category,sales,quantity,discount,order_priority,shipping_cost
0,1/1/2011,3,0,255,1,0,1,408,2,0.0,3,35.46
1,1/1/2011,3,0,702,0,9,1,120,3,0.1,3,9.72
2,1/1/2011,2,0,175,3,5,1,66,4,0.0,1,8.17
3,1/1/2011,2,2,939,4,7,1,45,3,0.5,1,4.82
4,1/1/2011,3,0,702,0,9,0,114,5,0.1,3,4.70
...,...,...,...,...,...,...,...,...,...,...,...,...
51285,31-12-2014,3,1,192,6,12,1,14,2,0.2,3,0.89
51286,31-12-2014,3,0,923,1,0,1,4,1,0.0,3,0.49
51287,31-12-2014,2,0,599,5,3,1,26,3,0.0,3,0.35
51288,31-12-2014,3,0,230,5,7,1,7,1,0.0,3,0.20
